In [ ]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 6.3 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving links from clirnet.docx to links from clirnet.docx


In [ ]:
from docx import Document

In [ ]:
doc = Document("/content/links from clirnet.docx")
links = []
for para in doc.paragraphs:
    text = para.text.strip()
    if text.startswith("https"):
        links.append(text)

print("Extracted links:", links)

Extracted links: ['https://en.wikipedia.org/wiki/Sustainable_agriculture', 'https://www.nature.com/articles/d41586-025-03353-5', 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10457221/', 'https://www.fao.org/3/y4671e/y4671e06.htm', 'https://www.medscape.com/viewarticle/time-reconsider-tramadol-chronic-pain-2025a1000ria', 'https://www.frontiersin.org/news/2025/09/01/rectangle-telescope-finding-habitable-planets', 'https://www.medscape.com/viewarticle/second-dose-boosts-shingles-protection-adults-aged-65-years-2025a1000ro7', 'https://www.theguardian.com/global-development/2025/oct/13/astro-ambassadors-stargazers-himalayas-hanle-ladakh-india']


In [ ]:
!pip install python-docx requests beautifulsoup4 pandas dspy-ai transformers OpenAI pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.2/285.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 112.5 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.76.0
    Uninstalling grpcio-1.76.0:
      Successfully uninstalled grpcio-1.76.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires grpcio>=1.71.2, but you have grpcio 1.67.1 which is incompatible.


In [ ]:
from docx import Document
import requests
from transformers import pipeline
from bs4 import BeautifulSoup
import pandas as pd
import dspy
from dspy.adapters import XMLAdapter
from typing import List
from pydantic import BaseModel, Field

In [ ]:
main_lm = dspy.LM("openai/gpt-4o-mini", api_key="REDACTED_API_KEY")
dspy.configure(lm=main_lm)
dspy.settings.configure(lm=main_lm, adapter=dspy.XMLAdapter())


In [ ]:
def scrape_text(url):
    try:
        r = requests.get(url, timeout=15)
        soup = BeautifulSoup(r.text, "lxml")
        paragraphs = [p.get_text(strip=True) for p in soup.find_all("p")]
        return " ".join(paragraphs)
    except Exception:
        return ""

rows = [{"link": link, "raw_text": scrape_text(link)} for link in links]
df_raw = pd.DataFrame(rows)

In [ ]:
class CatchyTagSignature(dspy.Signature):
    text = dspy.InputField(desc="The raw article text")
    tag = dspy.OutputField(desc="Catchy 1–2 word tag taken from the text")

class CatchyTagClassifier(dspy.Module):
    def __init__(self):
        super().__init__()
        self.predict = dspy.Predict(CatchyTagSignature)

    def forward(self, text):
        return self.predict(text=text).tag

classifier = CatchyTagClassifier()

In [ ]:
def refine_tag(text, max_attempts=3):
    attempt = 0
    tag = ""
    while attempt < max_attempts:
        tag = classifier(text)
        if tag.lower() in ["general", "article", "news", "misc", "other"]:
            attempt += 1
            text = text + "\n\nFocus: Extract a specific 1–2 word phrase from this text."
        else:
            break

    if tag.lower() in ["general", "article", "news", "misc", "other"] or not tag.strip():
        words = [w for w in text.split() if len(w) > 4][:2]
        tag = " ".join(words) if words else "Topic"

    return tag


In [ ]:
 class EntityTag(BaseModel):
    tag_type: str = Field(description="catchy topic name or entity extracted from text")
    attr_type: str = Field(description="most relevant semantic type (Drug, Disease, Symptom, etc.)")

class ExtractEntities(dspy.Signature):
    paragraph: str = dspy.InputField(desc="input paragraph")
    entity: EntityTag = dspy.OutputField(desc="single most relevant entity with tag_type + attr_type")

extractor = dspy.Predict(ExtractEntities)


In [ ]:
predicted_tags = []
entity_tags = []

for text in df_raw["raw_text"]:
    if not text.strip():
        # Fallback if article text is empty
        predicted_tags.append(refine_tag("Fallback text"))
        entity_tags.append({"tag_type": "None", "attr_type": "None"})
    else:
        # Generate refined tag
        predicted_tags.append(refine_tag(text))

        # Extract most relevant entity
        result = extractor(paragraph=text)
        try:
            entity_tags.append({
                "tag_type": result.entity.tag_type,
                "attr_type": result.entity.attr_type
            })
        except Exception:
            # Robust fallback in case structured output fails
            entity_tags.append({"tag_type": "Keyword", "attr_type": "Unknown"})


2025/12/09 18:10:51 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


RateLimitError: litellm.RateLimitError: RateLimitError: OpenAIException - Rate limit reached for gpt-4o-mini in organization org-C6BlerjG4FciVCCruxwM9h0t on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.

In [ ]:
df_result = pd.DataFrame({
    "link": df_raw["link"],
    "tag": predicted_tags,
    "tag_type": [e["tag_type"] for e in entity_tags],
    "attr_type": [e["attr_type"] for e in entity_tags]
})

# Step 7: Export to CSV
df_result.to_csv("link_tag_entities.csv", index=False)

print(df_result.head(10))

ValueError: array length 5 does not match index length 8

In [ ]:
df_result = pd.DataFrame({
    "link": df_raw["link"],
    "tag": predicted_tags,
    "tag_type": [e["tag_type"] for e in entity_tags],
    "attr_type": [e["attr_type"] for e in entity_tags]
})

print(df_result.head(10))

ValueError: All arrays must be of the same length